## Crimes by municipality Mexico
Focus crimes: Secuestro, Extorsión, Robo a negocio


In [459]:
# Importing libraries
import pandas as pd 
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt 
import seaborn as sns
import regex as re

%matplotlib inline

### 1.- Importing data

In [460]:
# Database with crimes information
df = pd.read_csv('data/delitos-datos-abiertos.csv')
# Database with population information
pop = pd.read_csv('data/poblaciones_2015.csv', skiprows=4, encoding='latin-1', skipfooter=6, engine='python')
# Database with cp information
cp = pd.read_excel('data/codigos-postales-mexico.xlsx')

# Formatting column names
df.columns = [x.lower().strip() for x in df.columns]
pop.columns = [x.lower().strip() for x in pop.columns]
cp.columns = [x.lower().strip() for x in cp.columns]

# Formatting date columns
df['fecha'] = pd.to_datetime(df['fecha'], infer_datetime_format=True)

# Renaming columns
df.rename(columns={'inegi_entidad': 'id_entidad',
                    'inegi_municipio': 'id_municipio'}, inplace=True)

pop.drop(columns='id_municipio', inplace=True)

pop.rename(columns={'estado': 'entidad',
                    'cve_inegi': 'id_municipio',
                    'id_estado': 'id_entidad'}, inplace=True)
                    
cp.rename(columns={'estado': 'entidad'}, inplace=True)

df.head()

,id_entidad,entidad,id_municipio,municipio,id_delito,delito,carpetas,tasa,fecha
0,1,Aguascalientes,1001,Aguascalientes,1100,Homicidio doloso,3,0.323526,2021-01-01
1,1,Aguascalientes,1002,Asientos,1100,Homicidio doloso,0,0.000000,2021-01-01
2,1,Aguascalientes,1010,El Llano,1100,Homicidio doloso,0,0.000000,2021-01-01
3,1,Aguascalientes,1009,Tepezala,1100,Homicidio doloso,0,0.000000,2021-01-01
4,1,Aguascalientes,1007,Rincon De Romos,1100,Homicidio doloso,0,0.000000,2021-01-01


In [461]:
pop.head()

,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total
0,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190
1,Aguascalientes,Asientos,1002,1,22745,23719,46464
2,Aguascalientes,Calvillo,1003,1,27298,28750,56048
3,Aguascalientes,Cosío,1004,1,7552,8025,15577
4,Aguascalientes,Jesús María,1005,1,60135,60270,120405


In [462]:
df.tail()

,id_entidad,entidad,id_municipio,municipio,id_delito,delito,carpetas,tasa,fecha
471739,32,Zacatecas,32047,Teul De Gonzalez Ortega,9000,Lesiones dolosas,1,17.430714,2021-12-01
471740,32,Zacatecas,32007,Concepcion Del Oro,9000,Lesiones dolosas,1,7.334067,2021-12-01
471741,32,Zacatecas,32008,Cuauhtemoc,9000,Lesiones dolosas,0,0.000000,2021-12-01
471742,32,Zacatecas,32016,General Panfilo Natera,9000,Lesiones dolosas,1,4.203977,2021-12-01
471743,32,Zacatecas,32056,Zacatecas,9000,Lesiones dolosas,23,15.266466,2021-12-01


In [463]:
pop.tail()

,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total
2452,Zacatecas,Villa Hidalgo,32054,32,9433,9722,19155
2453,Zacatecas,Villanueva,32055,32,14793,15447,30240
2454,Zacatecas,Zacatecas,32056,32,70855,75292,146147
2455,Zacatecas,Trancoso,32057,32,9505,9908,19413
2456,Zacatecas,Santa María de la Paz *,32058,32,1305,1351,2656


In [464]:
cp.head()

,código,asentamiento,tipo,municipio,ciudad,entidad
0,1000,San Angel,Colonia,Álvaro Obregón,Ciudad de México,Distrito Federal
1,1010,Los Alpes,Colonia,Álvaro Obregón,Ciudad de México,Distrito Federal
2,1020,Guadalupe Inn,Colonia,Álvaro Obregón,Ciudad de México,Distrito Federal
3,1028,Secretaria de Contraloría y Desarrollo Adminis...,Gran usuario,Álvaro Obregón,Ciudad de México,Distrito Federal
4,1029,INFONAVIT,Gran usuario,Álvaro Obregón,Ciudad de México,Distrito Federal


### 1.1 Cleaning data

In [465]:
# Cleaning  * from names in pop df
pop['municipio'] = pop['municipio'].replace(' *', '', regex=True)
pop['municipio'] = pop['municipio'].replace('\*', '', regex=True)

# Changing Distrito Federal for CDMX
pop['entidad'] = pop['entidad'].replace('Distrito Federal', 'Ciudad de México')
df['entidad'] = df['entidad'].replace('Distrito Federal', 'Ciudad de México')
cp['entidad'] = cp['entidad'].replace('Distrito Federal', 'Ciudad de México')

pop.tail()

,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total
2452,Zacatecas,VillaHidalgo,32054,32,9433,9722,19155
2453,Zacatecas,Villanueva,32055,32,14793,15447,30240
2454,Zacatecas,Zacatecas,32056,32,70855,75292,146147
2455,Zacatecas,Trancoso,32057,32,9505,9908,19413
2456,Zacatecas,SantaMaríadelaPaz,32058,32,1305,1351,2656


In [466]:
# Separating words in pop df
string = 'VillaHidalgo'
words = re.findall('[A-Z][a-z]*', string)
string2 = ' '.join(words)
print(string2)

# Deleting spaces
string = 'Villa Hidalgo'
string3 = string.replace(' ', '')
string3

Villa Hidalgo


'VillaHidalgo'

In [467]:
# Applying functions
# pop['municipio_sep'] = pop['municipio'].apply(lambda x: ' '.join(re.findall('[A-Z][a-zÀ-ÿ]*', x)))
cp['municipio_strip'] = cp['municipio'].apply(lambda x: x.replace(' ', ''))

In [468]:
cp

,código,asentamiento,tipo,municipio,ciudad,entidad,municipio_strip
0,1000,San Angel,Colonia,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón
1,1010,Los Alpes,Colonia,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón
2,1020,Guadalupe Inn,Colonia,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón
3,1028,Secretaria de Contraloría y Desarrollo Adminis...,Gran usuario,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón
4,1029,INFONAVIT,Gran usuario,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón
...,...,...,...,...,...,...,...
143219,99993,Cuxpala,Pueblo,Moyahua de Estrada,NaN,Zacatecas,MoyahuadeEstrada
143220,99994,Vicente Guerrero,Pueblo,Moyahua de Estrada,NaN,Zacatecas,MoyahuadeEstrada
143221,99998,Palmarejo,Ranchería,Moyahua de Estrada,NaN,Zacatecas,MoyahuadeEstrada
143222,99998,Jesús Maria,Ranchería,Moyahua de Estrada,NaN,Zacatecas,MoyahuadeEstrada


### 2.- Exploring data

#### 2.1.- Exploring datatypes

In [469]:
df.dtypes

id_entidad               int64
entidad                 object
id_municipio             int64
municipio               object
id_delito                int64
delito                  object
carpetas                 int64
tasa                   float64
fecha           datetime64[ns]
dtype: object

In [470]:
pop.dtypes

entidad         object
municipio       object
id_municipio     int64
id_entidad       int64
hombres          int64
mujeres          int64
total            int64
dtype: object

In [471]:
cp.dtypes

código              int64
asentamiento       object
tipo               object
municipio          object
ciudad             object
entidad            object
municipio_strip    object
dtype: object

#### 2.2.- Cleaning data to merge

Checking id_municipio

In [472]:
# Creating list of unique ids in each dataset
unique_id_df = df['id_municipio'].unique()
unique_id_pop = pop['id_municipio'].unique()

# Len of the lists
len_id_df = len(df['id_municipio'].unique())
len_id_pop = len(pop['id_municipio'].unique())

print(len_id_df)
print(len_id_pop)

for i in unique_id_df:
    if i not in unique_id_pop:
        m = df[df['id_municipio'] == i].iloc[0]['municipio']
        print('id {} ({}) does not exist in pop data'.format(i, m))

print('\n')


2457
2457
id 7101 (Tuxtla Gutierrez) does not exist in pop data
id 7109 (Yajalon) does not exist in pop data
id 7110 (San Lucas) does not exist in pop data
id 7119 (Santiago El Pinar) does not exist in pop data
id 7114 (Benemerito De Las Americas) does not exist in pop data
id 7107 (Villa Corzo) does not exist in pop data
id 7117 (Montecristo De Guerrero) does not exist in pop data
id 7105 (Union Juarez) does not exist in pop data
id 7111 (Zinacantan) does not exist in pop data
id 7118 (San Andres Duraznal) does not exist in pop data
id 7102 (Tuxtla Chico) does not exist in pop data
id 7116 (Marques De Comillas) does not exist in pop data
id 7100 (Tumbala) does not exist in pop data
id 7108 (Villaflores) does not exist in pop data
id 7103 (Tuzantan) does not exist in pop data
id 7104 (Tzimol) does not exist in pop data
id 7113 (Aldama) does not exist in pop data
id 7112 (San Juan Cancuc) does not exist in pop data
id 7115 (Maravilla Tenejapa) does not exist in pop data
id 7106 (Venusti

In [473]:
for i in unique_id_pop:
    if i not in unique_id_df:
        m = pop[pop['id_municipio'] == i].iloc[0]['municipio']
        print('id {} ({}) does not exist in crimes data'.format(i, m))

id 70100 (Tumbalá) does not exist in crimes data
id 70101 (TuxtlaGutiérrez) does not exist in crimes data
id 70102 (TuxtlaChico) does not exist in crimes data
id 70103 (Tuzantán) does not exist in crimes data
id 70104 (Tzimol) does not exist in crimes data
id 70105 (UniónJuárez) does not exist in crimes data
id 70106 (VenustianoCarranza) does not exist in crimes data
id 70107 (VillaCorzo) does not exist in crimes data
id 70108 (Villaflores) does not exist in crimes data
id 70109 (Yajalón) does not exist in crimes data
id 70110 (SanLucas) does not exist in crimes data
id 70111 (Zinacantán) does not exist in crimes data
id 70112 (SanJuanCancuc) does not exist in crimes data
id 70113 (Aldama) does not exist in crimes data
id 70114 (BeneméritodelasAméricas) does not exist in crimes data
id 70115 (MaravillaTenejapa) does not exist in crimes data
id 70116 (MarquésdeComillas) does not exist in crimes data
id 70117 (MontecristodeGuerrero) does not exist in crimes data
id 70118 (SanAndrésDurazn

In [474]:
# Fixing misslabeled data
for i in unique_id_pop:
    if i not in unique_id_df:
        to_replace_with = i - 70000 + 7000
        print('{} replaced with {}'.format(i, to_replace_with))
        pop['id_municipio'] = pop['id_municipio'].replace(i, to_replace_with)

70100 replaced with 7100
70101 replaced with 7101
70102 replaced with 7102
70103 replaced with 7103
70104 replaced with 7104
70105 replaced with 7105
70106 replaced with 7106
70107 replaced with 7107
70108 replaced with 7108
70109 replaced with 7109
70110 replaced with 7110
70111 replaced with 7111
70112 replaced with 7112
70113 replaced with 7113
70114 replaced with 7114
70115 replaced with 7115
70116 replaced with 7116
70117 replaced with 7117
70118 replaced with 7118
70119 replaced with 7119


In [475]:
# Creating list of unique ids in each dataset
unique_id_df = df['id_municipio'].unique()
unique_id_pop = pop['id_municipio'].unique()

# Len of the lists
len_id_df = len(df['id_municipio'].unique())
len_id_pop = len(pop['id_municipio'].unique())

for i in unique_id_pop:
    if i not in unique_id_df:
        print('id {} does not exist in crimes data'.format(i))

for i in unique_id_df:
    if i not in unique_id_pop:
        print('id {} does not exist in crimes data'.format(i))

print('No missing values')

if len_id_df == len_id_pop:
    print('Equal lenghts')

print('Ready to merge crimes and pop')


No missing values
Equal lenghts
Ready to merge crimes and pop


Checking municipio column

In [476]:
# Creating list of unique ids in each dataset
unique_mun_cp = cp['municipio_strip'].unique()
unique_mun_pop = pop['municipio'].unique()

# Len of the lists
len_mun_cp = len(cp['municipio_strip'].unique())
len_mun_pop = len(pop['municipio'].unique())

print(len_mun_cp)
print(len_mun_pop)
print(len_mun_pop - len_mun_cp)

print('\n')

for i in unique_mun_cp:
    if i not in unique_mun_pop:
        print('Municipio {}  does not exist in pop data'.format(i))



2318
2317
-1


Municipio TezoatlándeSegurayLuna  does not exist in pop data
Municipio SanJuanMixtepec-Dto.08-  does not exist in pop data
Municipio SanJuanMixtepec-Dto.26-  does not exist in pop data
Municipio SanPedroMixtepec-Dto.26-  does not exist in pop data
Municipio SanPedroMixtepec-Dto.22-  does not exist in pop data
Municipio Medellín  does not exist in pop data


In [477]:
# Cleaning -Dto.xx- from name in cp database
cp['municipio_strip'] = cp['municipio_strip'].replace('\-.+\..+', '', regex=True)

# Creating list of unique ids in each dataset
unique_mun_cp = cp['municipio_strip'].unique()
unique_mun_pop = pop['municipio'].unique()

# Len of the lists
len_mun_cp = len(cp['municipio_strip'].unique())
len_mun_pop = len(pop['municipio'].unique())

print(len_mun_cp)
print(len_mun_pop)
print(len_mun_pop - len_mun_cp)

print('\n')

for i in unique_mun_cp:
    if i not in unique_mun_pop:
        print('Municipio {}  does not exist in pop data'.format(i))

2316
2317
1


Municipio TezoatlándeSegurayLuna  does not exist in pop data
Municipio Medellín  does not exist in pop data


In [478]:
for i in unique_mun_pop:
    if i not in unique_mun_cp:
        print('Municipio {}  does not exist in cp data'.format(i))

Municipio ZacualpandeAmilpas  does not exist in cp data
Municipio HeroicaVillaTezoatlándeSegurayLuna,CunadelaIndependenciadeOaxaca  does not exist in cp data
Municipio MedellíndeBravo  does not exist in cp data


In [479]:
# Replacing values manually
pop['municipio'] = pop['municipio'].replace('HeroicaVillaTezoatlándeSegurayLuna,CunadelaIndependenciadeOaxaca',
                                            'TezoatlándeSegurayLuna')

pop['municipio'] = pop['municipio'].replace('MedellíndeBravo', 'Medellín')

In [480]:
# Creating list of unique ids in each dataset
unique_mun_cp = cp['municipio_strip'].unique()
unique_mun_pop = pop['municipio'].unique()

# Len of the lists
len_mun_cp = len(cp['municipio_strip'].unique())
len_mun_pop = len(pop['municipio'].unique())

print(len_mun_cp)
print(len_mun_pop)
print(len_mun_pop - len_mun_cp)

print('\n')

for i in unique_mun_pop:
    if i not in unique_mun_cp:
        print('Municipio {} does not exist in cp data'.format(i))

pop[pop['municipio'] == 'ZacualpandeAmilpas']

2316
2317
1


Municipio ZacualpandeAmilpas does not exist in cp data


,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total
925,Morelos,ZacualpandeAmilpas,17032,17,4550,4820,9370


In [481]:
# Correcting last value
pop['municipio'] = pop['municipio'].replace('ZacualpandeAmilpas', 'Zacualpan')

In [482]:
# Creating list of unique ids in each dataset
unique_mun_cp = cp['municipio_strip'].unique()
unique_mun_pop = pop['municipio'].unique()

# Len of the lists
len_mun_cp = len(cp['municipio_strip'].unique())
len_mun_pop = len(pop['municipio'].unique())

for i in unique_mun_pop:
    if i not in unique_mun_cp:
        print('Municipio {} does not exist in cp data'.format(i))

for i in unique_mun_cp:
    if i not in unique_mun_pop:
        print('Municipio {} does not exist in cp data'.format(i))

print('No missing values')

if len_id_df == len_id_pop:
    print('Equal lenghts')

print('Ready to merge crimes and pop')


No missing values
Equal lenghts
Ready to merge crimes and pop


Checking entidad column

In [483]:
# Creating list of unique ids in each dataset
unique_mun_cp = cp['entidad'].unique()
unique_mun_pop = pop['entidad'].unique()

# Len of the lists
len_mun_cp = len(cp['entidad'].unique())
len_mun_pop = len(pop['entidad'].unique())

for i in unique_mun_pop:
    if i not in unique_mun_cp:
        print('entidad {} does not exist in cp data'.format(i))

for i in unique_mun_cp:
    if i not in unique_mun_pop:
        print('entidad {} does not exist in pop data'.format(i))

print('No missing values')

if len_id_df == len_id_pop:
    print('Equal lenghts')

print('Ready to merge crimes and pop')


No missing values
Equal lenghts
Ready to merge crimes and pop


### 3.- Merging

#### 3.1 Merging zip code and population data

In [484]:
# Generating column entidad-municipio for pop
pop['entidad_municipio'] = pop['entidad'] + '-' + pop['municipio']

# Generating column entidad-municipio for cp
cp['entidad_municipio'] = cp['entidad'] + '-' + cp['municipio_strip']
cp.head()

,código,asentamiento,tipo,municipio,ciudad,entidad,municipio_strip,entidad_municipio
0,1000,San Angel,Colonia,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón,Ciudad de México-ÁlvaroObregón
1,1010,Los Alpes,Colonia,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón,Ciudad de México-ÁlvaroObregón
2,1020,Guadalupe Inn,Colonia,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón,Ciudad de México-ÁlvaroObregón
3,1028,Secretaria de Contraloría y Desarrollo Adminis...,Gran usuario,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón,Ciudad de México-ÁlvaroObregón
4,1029,INFONAVIT,Gran usuario,Álvaro Obregón,Ciudad de México,Ciudad de México,ÁlvaroObregón,Ciudad de México-ÁlvaroObregón


In [485]:
df.head()

,id_entidad,entidad,id_municipio,municipio,id_delito,delito,carpetas,tasa,fecha
0,1,Aguascalientes,1001,Aguascalientes,1100,Homicidio doloso,3,0.323526,2021-01-01
1,1,Aguascalientes,1002,Asientos,1100,Homicidio doloso,0,0.000000,2021-01-01
2,1,Aguascalientes,1010,El Llano,1100,Homicidio doloso,0,0.000000,2021-01-01
3,1,Aguascalientes,1009,Tepezala,1100,Homicidio doloso,0,0.000000,2021-01-01
4,1,Aguascalientes,1007,Rincon De Romos,1100,Homicidio doloso,0,0.000000,2021-01-01


In [486]:
pop.head()

,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total,entidad_municipio
0,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,Aguascalientes-Aguascalientes
1,Aguascalientes,Asientos,1002,1,22745,23719,46464,Aguascalientes-Asientos
2,Aguascalientes,Calvillo,1003,1,27298,28750,56048,Aguascalientes-Calvillo
3,Aguascalientes,Cosío,1004,1,7552,8025,15577,Aguascalientes-Cosío
4,Aguascalientes,JesúsMaría,1005,1,60135,60270,120405,Aguascalientes-JesúsMaría


In [487]:
# Merging pop and cp data
cols_cp = ['código', 'asentamiento', 'tipo', 'municipio', 'entidad_municipio']

cp_pop = pd.merge(cp[cols_cp], pop, how='left', left_on='entidad_municipio', right_on='entidad_municipio')
cp_pop.head()

,código,asentamiento,tipo,municipio_x,entidad_municipio,entidad,municipio_y,id_municipio,id_entidad,hombres,mujeres,total
0,1000,San Angel,Colonia,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
1,1010,Los Alpes,Colonia,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
2,1020,Guadalupe Inn,Colonia,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
3,1028,Secretaria de Contraloría y Desarrollo Adminis...,Gran usuario,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
4,1029,INFONAVIT,Gran usuario,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982


In [488]:
len(cp) -len(cp_pop)

-213

In [489]:
# Difference comes from Districts different cp
cp[cp['municipio'].str.contains('\-.+\..+', regex=True)]

,código,asentamiento,tipo,municipio,ciudad,entidad,municipio_strip,entidad_municipio
91558,69770,Las Flores,Colonia,San Juan Mixtepec -Dto. 08 -,NaN,Oaxaca,SanJuanMixtepec,Oaxaca-SanJuanMixtepec
91559,69770,San Miguel Lado,Barrio,San Juan Mixtepec -Dto. 08 -,NaN,Oaxaca,SanJuanMixtepec,Oaxaca-SanJuanMixtepec
91560,69770,San Juan Mixtepec - Dto.08 Centro,Colonia,San Juan Mixtepec -Dto. 08 -,NaN,Oaxaca,SanJuanMixtepec,Oaxaca-SanJuanMixtepec
91561,69770,San Pedro Calvario,Barrio,San Juan Mixtepec -Dto. 08 -,NaN,Oaxaca,SanJuanMixtepec,Oaxaca-SanJuanMixtepec
91562,69770,De Jesús,Barrio,San Juan Mixtepec -Dto. 08 -,NaN,Oaxaca,SanJuanMixtepec,Oaxaca-SanJuanMixtepec
...,...,...,...,...,...,...,...,...
95508,71998,El Salitre,Ranchería,San Pedro Mixtepec -Dto. 22 -,NaN,Oaxaca,SanPedroMixtepec,Oaxaca-SanPedroMixtepec
95509,71998,Cerro Zopilote,Ranchería,San Pedro Mixtepec -Dto. 22 -,NaN,Oaxaca,SanPedroMixtepec,Oaxaca-SanPedroMixtepec
95510,71998,El Toledo,Ranchería,San Pedro Mixtepec -Dto. 22 -,NaN,Oaxaca,SanPedroMixtepec,Oaxaca-SanPedroMixtepec
95511,71998,Regadío,Ranchería,San Pedro Mixtepec -Dto. 22 -,NaN,Oaxaca,SanPedroMixtepec,Oaxaca-SanPedroMixtepec


In [490]:
pop[pop['municipio'].str.contains('Mixtepec', regex=True)]

,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total,entidad_municipio
1045,Oaxaca,MagdalenaMixtepec,20048,20,662,713,1375,Oaxaca-MagdalenaMixtepec
1120,Oaxaca,SanBernardoMixtepec,20123,20,1313,1428,2741,Oaxaca-SanBernardoMixtepec
1150,Oaxaca,SanGabrielMixtepec,20153,20,2355,2481,4836,Oaxaca-SanGabrielMixtepec
1205,Oaxaca,SanJuanMixtepec,20208,20,3113,3567,6680,Oaxaca-SanJuanMixtepec
1206,Oaxaca,SanJuanMixtepec,20209,20,285,375,660,Oaxaca-SanJuanMixtepec
1268,Oaxaca,SanMiguelMixtepec,20271,20,1295,1349,2644,Oaxaca-SanMiguelMixtepec
1315,Oaxaca,SanPedroMixtepec,20318,20,23381,24955,48336,Oaxaca-SanPedroMixtepec
1316,Oaxaca,SanPedroMixtepec,20319,20,523,551,1074,Oaxaca-SanPedroMixtepec
1375,Oaxaca,SantaCruzMixtepec,20378,20,1624,1801,3425,Oaxaca-SantaCruzMixtepec


In [491]:
# Dropping second repeated municipios
pop.drop([1206, 1316], inplace=True)
pop[pop['municipio'].str.contains('Mixtepec', regex=True)]

,entidad,municipio,id_municipio,id_entidad,hombres,mujeres,total,entidad_municipio
1045,Oaxaca,MagdalenaMixtepec,20048,20,662,713,1375,Oaxaca-MagdalenaMixtepec
1120,Oaxaca,SanBernardoMixtepec,20123,20,1313,1428,2741,Oaxaca-SanBernardoMixtepec
1150,Oaxaca,SanGabrielMixtepec,20153,20,2355,2481,4836,Oaxaca-SanGabrielMixtepec
1205,Oaxaca,SanJuanMixtepec,20208,20,3113,3567,6680,Oaxaca-SanJuanMixtepec
1268,Oaxaca,SanMiguelMixtepec,20271,20,1295,1349,2644,Oaxaca-SanMiguelMixtepec
1315,Oaxaca,SanPedroMixtepec,20318,20,23381,24955,48336,Oaxaca-SanPedroMixtepec
1375,Oaxaca,SantaCruzMixtepec,20378,20,1624,1801,3425,Oaxaca-SantaCruzMixtepec


In [492]:
# Merging again
# Merging pop and cp data
cols_cp = ['código', 'asentamiento', 'tipo', 'municipio', 'entidad_municipio']

cp_pop = pd.merge(cp[cols_cp], pop, how='left', left_on='entidad_municipio', right_on='entidad_municipio')
if len(cp) -len(cp_pop) == 0:
    print('Same shapes!')

cp_pop.rename(columns={'municipio_x': 'municipio', 'municipio_y': 'municipio_strip'}, inplace=True)
cp_pop.head()

Same shapes!


,código,asentamiento,tipo,municipio,entidad_municipio,entidad,municipio_strip,id_municipio,id_entidad,hombres,mujeres,total
0,1000,San Angel,Colonia,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
1,1010,Los Alpes,Colonia,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
2,1020,Guadalupe Inn,Colonia,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
3,1028,Secretaria de Contraloría y Desarrollo Adminis...,Gran usuario,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982
4,1029,INFONAVIT,Gran usuario,Álvaro Obregón,Ciudad de México-ÁlvaroObregón,Ciudad de México,ÁlvaroObregón,9010,9,355754,394228,749982


In [493]:
# Exporting to csv
cp_pop.to_excel('data/cp-population.xlsx')

#### 3.2.- Merging new dataframe with criminality dataframe

In [494]:
merged = pd.merge(cp_pop, df, on='id_municipio', how='left')

In [495]:
merged[merged['id_municipio'] == 1001]

,código,asentamiento,tipo,municipio_x,entidad_municipio,entidad_x,municipio_strip,id_municipio,id_entidad_x,hombres,mujeres,total,id_entidad_y,entidad_y,municipio_y,id_delito,delito,carpetas,tasa,fecha
400896,20000,Zona Centro,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,1100,Homicidio doloso,3,0.323526,2021-01-01
400897,20000,Zona Centro,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,1120,Feminicidio,2,0.418447,2021-01-01
400898,20000,Zona Centro,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,1200,Homicidio culposo,5,0.539210,2021-01-01
400899,20000,Zona Centro,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,2000,Secuestro,0,0.000000,2021-01-01
400900,20000,Zona Centro,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,3000,Extorsión,5,0.539210,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542011,20399,El Turicate,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,5000,Violación,25,2.696052,2021-12-01
542012,20399,El Turicate,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,6000,Violencia familiar,116,12.509679,2021-12-01
542013,20399,El Turicate,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,7000,Trata de personas,0,0.000000,2021-12-01
542014,20399,El Turicate,Colonia,Aguascalientes,Aguascalientes-Aguascalientes,Aguascalientes,Aguascalientes,1001,1,425731,451459,877190,1,Aguascalientes,Aguascalientes,8000,Narcomenudeo,17,1.833315,2021-12-01


In [496]:
# Grouping population information by id_municipio
_ = cp_pop.groupby('id_municipio').agg({'hombres': np.mean,
                                    'mujeres': np.mean,
                                    'total': np.mean})

# Joining
df_pop = pd.merge(df, _, on='id_municipio', how='left')

In [497]:
df_pop

,id_entidad,entidad,id_municipio,municipio,id_delito,delito,carpetas,tasa,fecha,hombres,mujeres,total
0,1,Aguascalientes,1001,Aguascalientes,1100,Homicidio doloso,3,0.323526,2021-01-01,425731.0,451459.0,877190.0
1,1,Aguascalientes,1002,Asientos,1100,Homicidio doloso,0,0.000000,2021-01-01,22745.0,23719.0,46464.0
2,1,Aguascalientes,1010,El Llano,1100,Homicidio doloso,0,0.000000,2021-01-01,9982.0,10263.0,20245.0
3,1,Aguascalientes,1009,Tepezala,1100,Homicidio doloso,0,0.000000,2021-01-01,10197.0,10729.0,20926.0
4,1,Aguascalientes,1007,Rincon De Romos,1100,Homicidio doloso,0,0.000000,2021-01-01,26693.0,27173.0,53866.0
...,...,...,...,...,...,...,...,...,...,...,...,...
471739,32,Zacatecas,32047,Teul De Gonzalez Ortega,9000,Lesiones dolosas,1,17.430714,2021-12-01,2637.0,2741.0,5378.0
471740,32,Zacatecas,32007,Concepcion Del Oro,9000,Lesiones dolosas,1,7.334067,2021-12-01,6494.0,6450.0,12944.0
471741,32,Zacatecas,32008,Cuauhtemoc,9000,Lesiones dolosas,0,0.000000,2021-12-01,6037.0,6553.0,12590.0
471742,32,Zacatecas,32016,General Panfilo Natera,9000,Lesiones dolosas,1,4.203977,2021-12-01,11224.0,11968.0,23192.0


### 4.- Analysis

In [498]:
# Aggregating crimes by id_municipio
crimes_by_municipio = df_pop.groupby('id_municipio').agg({'carpetas': np.sum, 'total': np.mean})

# Generating crimes per 10000 people
crimes_by_municipio['crimes_10k'] = (crimes_by_municipio['carpetas'] / crimes_by_municipio['total']) * 10000

# Adding names of places
crimes_by_municipio = pd.merge(crimes_by_municipio, df[['entidad', 'municipio', 'id_municipio']],
                                how='left', on='id_municipio')
    


In [499]:
crimes_by_municipio = crimes_by_municipio.groupby('id_municipio').agg({'carpetas': np.sum, 'total': np.mean,
                                                                        'crimes_10k': np.mean, 'entidad': stats.mode,
                                                                        'municipio':stats.mode})

In [500]:
# Extracting only value without frequency
crimes_by_municipio['entidad'] = crimes_by_municipio['entidad'].apply(lambda x: x[0][0])
crimes_by_municipio['municipio'] = crimes_by_municipio['municipio'].apply(lambda x: x[0][0])


In [501]:
# Rearanging columns
crimes_by_municipio = crimes_by_municipio[['entidad', 'municipio', 'carpetas', 'total', 'crimes_10k']]

#### *Clean dataset aggregated by id_municipio*

In [502]:
# Clean dataset
crimes_by_municipio

,entidad,municipio,carpetas,total,crimes_10k
id_municipio,,,,,
1001,Aguascalientes,Aguascalientes,2129472,877190.0,126.437830
1002,Aguascalientes,Asientos,78912,46464.0,88.455579
1003,Aguascalientes,Calvillo,58368,56048.0,54.239224
1004,Aguascalientes,Cosio,28416,15577.0,95.011876
1005,Aguascalientes,Jesus Maria,287616,120405.0,124.413438
...,...,...,...,...,...
32054,Zacatecas,Villa Hidalgo,8640,19155.0,23.492561
32055,Zacatecas,Villanueva,33408,30240.0,57.539683
32056,Zacatecas,Zacatecas,372864,146147.0,132.879909


### X.- Coordinates data set

In [503]:
# Imoorting location lat, lon dataset
locations = pd.read_excel('data/coordenadas_municipios.xlsx')
locations.head()


,id_municipio,lat,lon
0,1001,21.879823,-102.296047
1,1002,22.238317,-102.089275
2,1003,21.846907,-102.718751
3,1004,22.366409,-102.300044
4,1005,21.961273,-102.343416


#### Coinciding id values

In [504]:
# Creating list of unique ids in each dataset
unique_id_crimes_by_municipio = crimes_by_municipio.index.unique()
unique_id_locations = locations['id_municipio'].unique()

# Len of the lists
len_id_crimes_by_municipio = len(crimes_by_municipio.index.unique())
len_id_locations = len(locations['id_municipio'].unique())

for i in unique_id_crimes_by_municipio:
    if i not in unique_id_locations:
        print('id_municipio {} does not exist in locations data'.format(i))

print('No missing values')

if len_id_df == len_id_locations:
    print('Equal lenghts')

print('Ready to merge crimes and locations')

No missing values
Ready to merge crimes and locations


#### Merging location information

In [505]:
crimes_by_municipio = crimes_by_municipio.merge(locations, how='left', on='id_municipio')
crimes_by_municipio

,id_municipio,entidad,municipio,carpetas,total,crimes_10k,lat,lon
0,1001,Aguascalientes,Aguascalientes,2129472,877190.0,126.437830,21.879823,-102.296047
1,1002,Aguascalientes,Asientos,78912,46464.0,88.455579,22.238317,-102.089275
2,1003,Aguascalientes,Calvillo,58368,56048.0,54.239224,21.846907,-102.718751
3,1004,Aguascalientes,Cosio,28416,15577.0,95.011876,22.366409,-102.300044
4,1005,Aguascalientes,Jesus Maria,287616,120405.0,124.413438,21.961273,-102.343416
...,...,...,...,...,...,...,...,...
2452,32054,Zacatecas,Villa Hidalgo,8640,19155.0,23.492561,22.357088,-101.712599
2453,32055,Zacatecas,Villanueva,33408,30240.0,57.539683,22.354259,-102.883726
2454,32056,Zacatecas,Zacatecas,372864,146147.0,132.879909,22.776096,-102.571836
2455,32057,Zacatecas,Trancoso,34176,19413.0,91.691135,22.735389,-102.366038


#### Creating scales

In [596]:
data = crimes_by_municipio
data = data.dropna()

limits = []
for x in range(0, 101, 20):
    limits.append(np.percentile(data['crimes_10k'], x))

data['class'] = data['crimes_10k'].apply(lambda x:
                                        0 if (x > limits[0] and x < limits[1])
                                        else
                                        (1 if (x > limits[1] and x < limits[2]) 
                                        else 
                                        (2 if (x > limits[2] and x < limits[3])
                                        else 
                                        (3 if (x > limits[3] and x < limits[4])
                                        else 4))))

<ipython-input-596-08e38a68d08a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['class'] = data['crimes_10k'].apply(lambda x:


In [597]:
data.head()

,id_municipio,entidad,municipio,carpetas,total,crimes_10k,lat,lon,class
0,1001,Aguascalientes,Aguascalientes,2129472,877190.0,126.437830,21.879823,-102.296047,4
1,1002,Aguascalientes,Asientos,78912,46464.0,88.455579,22.238317,-102.089275,4
2,1003,Aguascalientes,Calvillo,58368,56048.0,54.239224,21.846907,-102.718751,3
3,1004,Aguascalientes,Cosio,28416,15577.0,95.011876,22.366409,-102.300044,4
4,1005,Aguascalientes,Jesus Maria,287616,120405.0,124.413438,21.961273,-102.343416,4


#### Map

In [599]:
map = folium.Map(location = [20.693943, -100.985880], zoom_start=5.3)

data1 = data

for state in data1['entidad'].unique():
    data = data1[data1['entidad'] == state]

    colors = ['forestgreen', 'lime', 'yellow', 'lightcoral', 'firebrick']
    for cluster in range(5):
        color_cluster = colors[cluster]
        for point in range(len(data[data['class']==cluster])):
            folium.CircleMarker(location=[data.iloc[point]['lat'], data.iloc[point]['lon']],
            radius=8,
            color=color_cluster,
            fill_color=color_cluster,
            fill=True
            ).add_to(map)

data = data1

map.save('map_colors.html')
map

In [609]:
data[data['class'] == 4]
  

,id_municipio,entidad,municipio,carpetas,total,crimes_10k,lat,lon,class
0,1001,Aguascalientes,Aguascalientes,2129472,877190.0,126.437830,21.879823,-102.296047,4
1,1002,Aguascalientes,Asientos,78912,46464.0,88.455579,22.238317,-102.089275,4
3,1004,Aguascalientes,Cosio,28416,15577.0,95.011876,22.366409,-102.300044,4
4,1005,Aguascalientes,Jesus Maria,287616,120405.0,124.413438,21.961273,-102.343416,4
5,1006,Aguascalientes,Pabellon De Arteaga,106368,46473.0,119.209003,22.149202,-102.276542,4
...,...,...,...,...,...,...,...,...,...
2430,32032,Zacatecas,Morelos,24576,12354.0,103.610167,22.862194,-102.609535,4
2434,32036,Zacatecas,Ojocaliente,67584,43471.0,80.973523,22.571967,-102.252934,4
2447,32049,Zacatecas,Valparaiso,53376,32606.0,85.260382,22.771900,-103.567233,4
2454,32056,Zacatecas,Zacatecas,372864,146147.0,132.879909,22.776096,-102.571836,4


In [559]:
import folium
from folium.plugins import HeatMap

data['map_input'] = data.apply(lambda x: [x['lat'], x['lon'], x['crimes_10k']], axis=1)
map_data = data['map_input']

# Create map object
map = folium.Map(location = [20.693943, -100.985880], zoom_start=5.3)

HeatMap(map_data).add_to(map)

map.save('map1.html')

map

<ipython-input-559-9ac1f43ab5de>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['map_input'] = data.apply(lambda x: [x['lat'], x['lon'], x['crimes_10k']], axis=1)
